In [2]:
import pandas as pd
import re
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
df = pd.read_csv("../../datasets/Invoices_Std.csv", encoding='latin-1', low_memory=False)

In [7]:
CATEGORY_RULES = {
    "Trang trí nhà cửa": [
        r"t-light", r"lantern", r"candle", r"holder", r"heart", r"ornament",
        r"frame", r"cabinet", r"drawer", r"doormat", r"hanging", r"coat hanger",
        r"wood", r"sign", r"wall", r"storage tin", r"mirror", r"wreath",
        r"box", r"rack", r"pot", r"vase", r"bird", r"star", r"skull", r"hook",
        r"cushion", r"throw", r"felt", r"metal", r"ceramic", r"porcelain",
        r"shabby", r"chic", r"picture", r"shelves", r"display", r"garden",
        r"planter", r"statue", r"bamboo", r"tissue box", r"slipper", r"rug"
    ],
    "Nhà bếp": [
        r"mug", r"plate", r"cutlery", r"teaspoon", r"cake", r"tea towel",
        r"jam", r"recipe", r"snack box", r"lunch box", r"glass", r"cup",
        r"baking", r"coaster", r"bottle opener", r"apron", r"chopping board",
        r"jug", r"tray", r"kitchen", r"dish", r"salt and pepper", r"fork",
        r"knife", r"spoon", r"oil", r"vinegar", r"utensil", r"strainer",
        r"kettle", r"coffee"
    ],
    "Đồ chơi trẻ em": [
        r"playhouse", r"princess", r"doll", r"jigsaw", r"alphabet block",
        r"spaceboy", r"charlie", r"dinosaur", r"toy", r"game", r"puzzle",
        r"robot", r"tractor", r"plane", r"stickers", r"baby", r"kids",
        r"child", r"teddy", r"animal", r"miniature", r"colouring", r"clown",
        r"mermaid", r"wooden box"
    ],
    "Túi xách": [
        r"jumbo bag", r"shopper", r"storage bag", r"bag", r"basket", r"tote",
        r"pouch", r"sack", r"purse", r"wallet", r"suitcase", r"travel",
        r"holdall", r"kit bag", r"vanity case", r"luggage"
    ],
    "Quà tặng theo mùa": [
        r"christmas", r"gift", r"fairy", r"vintage", r"retro", r"bunny",
        r"easter", r"party", r"bunting", r"ribbon", r"birthday", r"celebration",
        r"new year", r"halloween", r"valentine", r"snowman", r"reindeer",
        r"santa", r"crackers"
    ],
    "Chăm sóc cơ thể": [
        r"hot water bottle", r"hand warmer", r"mug cosy", r"soap", r"bath",
        r"cosmetic", r"lotion", r"perfume", r"shampoo", r"cream", r"wash bag",
        r"sponge", r"towel", r"brush", r"manicure", r"face mask"
    ],
    "Văn phòng phẩm": [
        r"paper", r"card", r"paint set", r"sticker", r"clip", r"pen", r"pencil",
        r"notebook", r"diary", r"book", r"memo", r"postcard", r"journal",
        r"folder", r"file", r"stapler", r"eraser", r"rubber", r"ruler"
    ],
    "Thiết bị chiếu sáng": [
        r"night light", r"led", r"lamp", r"string lights", r"bulb", r"fairy lights"
    ],
    "Đồng hồ": [
        r"alarm clock", r"clock", r"watch", r"time"
    ],
    "Phụ kiện may mặc": [
        r"scarf", r"socks", r"cap", r"glove", r"jewel", r"ring", r"necklace",
        r"bracelet", r"hat", r"belt", r"tights", r"tie", r"brooch", r"pin",
        r"shoes", r"boot", r"sandal"
    ],
    "Vật liệu và Công cụ": [
        r"wire", r"tool", r"tape", r"roll", r"metal", r"glitter", r"glue",
        r"sewing", r"paint", r"kit"
    ],
    "Nội thất & Thiết bị lớn": [
        r"chair", r"table", r"bench", r"mattress", r"storage unit", r"trolley"
    ]
}

def infer_category(description):
    if not isinstance(description, str):
        return "Không xác định"
    
    text = description.lower()

    for category, patterns in CATEGORY_RULES.items():
        for p in patterns:
            if re.search(p, text):
                return category
    return "Không xác định"

df["Category"] = df["Description"].apply(infer_category)

In [13]:
basket_category = (
    df.groupby(['InvoiceNo', 'Category'])['Category']
      .count()
      .unstack()
      .fillna(0)
)

basket_category = basket_category > 0
basket_category = basket_category.astype(bool)

freq_cat = apriori(basket_category, min_support=0.01, use_colnames=True)
rules_cat = association_rules(freq_cat, metric="lift", min_threshold=1.0)
rules_cat = rules_cat[
    rules_cat.apply(
        lambda r: ("Không xác định" not in r["antecedents"]) and
                  ("Không xác định" not in r["consequents"]),
        axis=1
    )
]

top_rules = rules_cat[ (rules_cat['lift'] >= 1.5) & (rules_cat['confidence'] >= 0.1) ].sort_values('lift', ascending=False)
# print(top_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10).to_markdown(index=False))

def short_fs(fs):
    return ", ".join(list(fs))[:70]

tmp = top_rules.head(10).copy()
tmp["antecedents"] = tmp["antecedents"].apply(short_fs)
tmp["consequents"] = tmp["consequents"].apply(short_fs)

print(tmp[['antecedents','consequents','support','confidence','lift']].to_markdown(index=False))


| antecedents                                                            | consequents                                                            |   support |   confidence |    lift |
|:-----------------------------------------------------------------------|:-----------------------------------------------------------------------|----------:|-------------:|--------:|
| Chăm sóc cơ thể, Quà tặng theo mùa, Túi xách, Đồ chơi trẻ em           | Trang trí nhà cửa, Nhà bếp, Vật liệu và Công cụ, Văn phòng phẩm        | 0.0111071 |     0.222831 | 6.32449 |
| Trang trí nhà cửa, Nhà bếp, Vật liệu và Công cụ, Văn phòng phẩm        | Chăm sóc cơ thể, Quà tặng theo mùa, Túi xách, Đồ chơi trẻ em           | 0.0111071 |     0.315245 | 6.32449 |
| Vật liệu và Công cụ, Nhà bếp, Văn phòng phẩm                           | Chăm sóc cơ thể, Quà tặng theo mùa, Túi xách, Đồ chơi trẻ em           | 0.0111071 |     0.314433 | 6.30819 |
| Vật liệu và Công cụ, Nhà bếp, Văn phòng phẩm                           | 

In [6]:
def fast_top_products(df, cat_a, cat_b, limit_invoices=1000, limit_pairs=10):
    df_sub = df[df["Category"].isin([cat_a, cat_b])]

    inv = (
        df_sub.groupby("InvoiceNo")["Category"]
              .agg(lambda x: set(x))
    )

    inv_ok = inv[inv.apply(lambda s: {cat_a, cat_b}.issubset(s))].index[:limit_invoices]

    df_valid = df[df["InvoiceNo"].isin(inv_ok)]

    pairs = []

    for inv_no, group in df_valid.groupby("InvoiceNo"):
        products = list(group["Description"].unique())
        n = len(products)
        if n < 2:
            continue
        count = 0
        for i in range(n):
            for j in range(i+1, n):
                pairs.append((products[i], products[j]))
                count += 1
                if count >= limit_pairs:
                    break
            if count >= limit_pairs:
                break
        if len(pairs) >= limit_pairs:
            break

    result = pd.DataFrame(pairs, columns=["Sản phẩm", "Sản phẩm"])
    return result.head(limit_pairs)

results = {}

for _, r in top_rules.head(10).iterrows():
    a = list(r["antecedents"])[0]
    b = list(r["consequents"])[0]
    key = f"{a} -> {b}"
    results[key] = fast_top_products(df, a, b)

for name, tbl in results.items():
    print(name)
    print(tbl.to_markdown(index=False))
    print()


Chăm sóc cơ thể -> Trang trí nhà cửa
| Sản phẩm                           | Sản phẩm                            |
|:-----------------------------------|:------------------------------------|
| WHITE HANGING HEART T-LIGHT HOLDER | WHITE METAL LANTERN                 |
| WHITE HANGING HEART T-LIGHT HOLDER | CREAM CUPID HEARTS COAT HANGER      |
| WHITE HANGING HEART T-LIGHT HOLDER | KNITTED UNION FLAG HOT WATER BOTTLE |
| WHITE HANGING HEART T-LIGHT HOLDER | RED WOOLLY HOTTIE WHITE HEART.      |
| WHITE HANGING HEART T-LIGHT HOLDER | SET 7 BABUSHKA NESTING BOXES        |
| WHITE HANGING HEART T-LIGHT HOLDER | GLASS STAR FROSTED T-LIGHT HOLDER   |
| WHITE METAL LANTERN                | CREAM CUPID HEARTS COAT HANGER      |
| WHITE METAL LANTERN                | KNITTED UNION FLAG HOT WATER BOTTLE |
| WHITE METAL LANTERN                | RED WOOLLY HOTTIE WHITE HEART.      |
| WHITE METAL LANTERN                | SET 7 BABUSHKA NESTING BOXES        |

Trang trí nhà cửa -> Chăm sóc cơ thể
|